In [ ]:
import pandas as pd
import re


'declaring variables for calculating total rows, correct rows and wrong rows in six dimensions'
'Values for TASK 1'
totald11 = wrongd11 = correctd11 = 0
totald12 = wrongd12 = correctd12 = 0
totald21 = wrongd21 = correctd21 = 0
totald31 = wrongd31 = correctd31 = 0
totald32 = wrongd32 = correctd32 = 0
totald41 = wrongd41 = correctd41 = 0
totald61 = wrongd61 = correctd61 = 0
wrongdatad11 = pd.DataFrame()
wrongdatad12 = pd.DataFrame()
wrongdatad2 = pd.DataFrame()
wrongdatad31 = pd.DataFrame()
wrongdatad32 = pd.DataFrame()
doubledata_d4 = pd.DataFrame()
wrongdatad6 = pd.DataFrame()
types=pd.DataFrame()


'TASK 1 Functions'
def dimension1():
    'Dimension 1: Consistency'
    global totald11,wrongd11,correctd11,wrongdatad11
    global totald12,wrongd12,correctd12,wrongdatad12
    
    'Check1: The numbers of houses exist for every row'
    for line in data['House Number']:
        totald11 =totald11 + 1
        if pd.isnull(line):
            wrongd11 += 1
        else:
            correctd11 += 1
    wrongdatad11=pd.concat([wrongdatad11,data.loc[(data['House Number'].isnull())]])
    
    'Check2: "Registration State" exist for every row as two letters (example “CA”)'
    expr = re.compile(r'[A-Z][A-Z]')
    wronglist=[]
    for line in data['Registration State']:
        totald12 += 1
        if re.findall(r'[A-Z][A-Z]', line):
            correctd12 += 1
        else:
            wrongd12 += 1
            wronglist.append(line)
    wronglist=list(set(wronglist))                       
    
    for value in wronglist:
        wrongdatad12=pd.concat([wrongdatad12,data.loc[(data['Registration State']==value)]])
               
def dimension2():
    'Dimension2: Timeliness'
    global totald21,wrongd21,correctd21,wrongdatad2
    data['Issue Date'] = pd.to_datetime(data['Issue Date'])
    for line in data['Issue Date']:
        totald21 += 1
        if line.year==2017:
            correctd21 += 1
        else:
            wrongd21 += 1
    wrongdatad2=pd.concat([wrongdatad2,data.loc[(data['Issue Date']<'2017-01-01')]])
    wrongdatad2=pd.concat([wrongdatad2,data.loc[(data['Issue Date']>'2017-12-31')]])

def dimension3():
    'Dimension3: Completeness'
    global totald31,wrongd31,correctd31,wrongdatad31
    global totald32,wrongd32,correctd32,wrongdatad32
    
    'Check1: House where violation was captured is present'
    for line in data['House Number']:
        totald31 += 1
        if pd.isnull(line):
            wrongd31 += 1
        else:
            correctd31 += 1
    wrongdatad31=pd.concat([wrongdatad31,data.loc[(data['House Number'].isnull())]])
    
    'Check2: "Violation Description" should exist.'      
    for line in data['Violation Description']:
        totald32 += 1
        if pd.isnull(line):
            wrongd32 += 1
        else:
            correctd32 += 1
    wrongdatad32=pd.concat([wrongdatad32,data.loc[(data['Violation Description'].isnull())]])  
    
def dimension4():
    'Dimension4: Uniqueness: No double rows, no duplicates'
    global totald41,wrongd41,correctd41,doubledata_d4
    totald41+=data['Plate ID'].count()
    doubletemp=data[data.duplicated(['Violation County','Issue Date',
                                 'House Number','Street Name',
                                 'Violation Description','Plate ID',
                                 'Registration State'],keep='first')]
    doubledata_d4=pd.concat([doubledata_d4,doubletemp], ignore_index=True)
    
def dimension5():
    'Dimension5: Validity'
    global types
     
    if types.empty: #What for: function calculate only one 'chunk' iteration for memory economy.
        pass
    else:
        exit()
        
    types=pd.DataFrame()
    types['Column Name']=list(data)
    types['Type in dataset']=list(data.dtypes)
    types.loc[(types['Type in dataset'] =='object'), ['Type in dataset']] = 'plain text'
    types.loc[(types['Type in dataset'] =='int64'), ['Type in dataset']] = 'number'
    types.loc[(types['Type in dataset'] =='float64'), ['Type in dataset']] = 'number'
    types.loc[(types['Type in dataset'] =='datetime64[ns]'), ['Type in dataset']] = 'date'
    types['Business Requirements']='plain text'
    number=['Summons Number','Violation Code','Street Code1',
            'Street Code2','Street Code3','Vehicle Expiration Date',
            'Violation Precinct','Issuer Precinct', 'Issuer Code',
            'Date First Observed','Law Section','Vehicle Year',
            'Feet From Curb'
           ]
    types.loc[(types['Column Name'] =='Issue Date'), ['Business Requirements']] = 'date'
    for line in number:
        types.loc[(types['Column Name'] ==line), ['Business Requirements']] = 'number'
        
def dimension6():
    'Dimension6: Accuracy: "House Number" begin since a number'
    global totald61,wrongd61,correctd61,wrongdatad6
    expr = re.compile(r'^[0-9]{1,}')
    data['House Number']=data['House Number'].astype(str)
    for line in data['House Number']:
        totald61 += 1
        if re.findall(expr, line):
            correctd61 += 1
        else:
            wrongd61 += 1    
    wrongdatad6=pd.concat([wrongdatad6,data[data['House Number'].str.count('^[0-9]{1,}')==0]])  


'Main part: reading csv file with chunk'

url='/kaggle/input/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'            
for data in pd.read_csv(url, chunksize=500000):
    'Functions executing Task 1'
    dimension1()
    dimension2()
    dimension3()
    dimension4()
    dimension5()
    dimension6()
    
'Output TASK 1'
print('TASK 1\n\n')
'Output Dimension1: Consistency'
print('Dimension1: Consistency: empty values in column "House Number" ')
print('total rows:', totald11, 'correct rows:', correctd11, 'empty rows:', wrongd11)
print(wrongdatad11[['Summons Number','Plate ID','House Number']])
print('\n\nDimension1: wrong values in column "Registration State" ')
print('total rows:', totald12, 'correct rows:', correctd12, 'wrong rows:', wrongd12)
print(wrongdatad12[['Summons Number','Plate ID','Registration State']])

'Output Dimension2: Timeliness'
print('\n\nDimension2:Timeliness: incorrect data(the year must be only 2017)')
print('total rows:', totald21, 'correct rows:', correctd21, 'wrong rows:', wrongd21)
print(wrongdatad2[['Summons Number','Plate ID','Issue Date']])

'Output Dimension3: Completeness'
print('\n\nDimension3: Completeness: house where violation was captured is present')
print('total rows:', totald31, 'correct rows:', correctd31, 'wrong rows:', wrongd31)
print(wrongdatad31[['Summons Number','Plate ID','House Number']])
print('\n\nDimension3: Violation Description exist')
print('total rows:', totald32, 'correct rows:', correctd32, 'wrong rows:', wrongd32)
print(wrongdatad32[['Summons Number','Plate ID','Violation Description']])

'Output Dimension4: Uniqueness'
print('\n\nDimension4: Uniqueness: find double rows')
print('total rows:', totald41, 'double rows:', doubledata_d4['Plate ID'].count())
print(doubledata_d4[['Summons Number','Plate ID','Violation County','Issue Date',
                  'House Number','Street Name','Violation Description','Registration State']].head(10))
print(doubledata_d4['Plate ID'].count())

'Output Dimension5: Validity'
print('\n\nDimension5: Validity: incorrect data types')
print(types.loc[(types['Type in dataset'] != types['Business Requirements'])])
'Output: Check for outliers (deviations)'
print('\n\nCheck for outliers: Incorrect vehicle year(year must be not more than 2017):')
print(data.loc[(data['Vehicle Year'] > 2017),['Vehicle Year']])

'Output Dimension6: Accuracy'
print('\n\nDimension6: Accuracy: "House Number" begin since a number ')
print('total rows:', totald61, 'correct rows:', correctd61, 'wrong rows:', wrongd61)
print(wrongdatad6[['Summons Number','Plate ID','House Number']])